In [57]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Функция создания сетки
Создаем сетку по входным параметрам в виде двумерного массива
___
**параметры:** 
 - `n` — размер сетки (n x n)
 - `empty_ratio` — процентное соотношение пустых клеток
 - `blue_ratio` — процентное соотношение синих клеток
 - `red_ratio` — процентное соотношение красных клеток
___
*возвращает двумерный массив, где:*
 - 0 — пустая клетка
 - 1 — синяя клетка
 - 2 — красная клетка

In [58]:
def initialize_grid(n, empty_ratio, blue_ratio, red_ratio):
    num_cells = n * n
    num_empty = int(empty_ratio * num_cells)
    num_blue = int(blue_ratio * num_cells)
    num_red = num_cells - num_empty - num_blue
    grid = [0] * num_empty + [1] * num_blue + [2] * num_red
    random.shuffle(grid)
    return np.array(grid).reshape((n, n))

# Функция подсчета соседей
Считаем количество соседей клетки по координатам
___
**параметры:**
-  `grid` — двумерный массив клеток
-  `x` — координата x клетки
-  `y` — координата y клетки
___
*возвращает количество соседей одного цвета*

In [59]:
def count_same_neighbors(grid, x, y):
    n = grid.shape[0]
    color = grid[x, y]
    if color == 0:
        return 0  # пустая клетка

    # соседи
    neighbors = [
        (x - 1, y - 1), (x - 1, y), (x - 1, y + 1),
        (x, y - 1),             (x, y + 1),
        (x + 1, y - 1), (x + 1, y), (x + 1, y + 1)
    ]

    same_color_count = 0
    for nx, ny in neighbors:
        if 0 <= nx < n and 0 <= ny < n and grid[nx, ny] == color:
            same_color_count += 1

    return same_color_count


# Функция проверки клетки
Проверяем клетку на "счастье", возвращаем истину или ложь
___
**параметры:**
-  `grid` — двумерный массив клеток
-  `x` — координата x клетки
-  `y` — координата y клетки
- `threshold` — количество необходимых соседей 
___
*возвращает True/False*

In [60]:
def is_happy(grid, x, y, threshold):
    return count_same_neighbors(grid, x, y) >= threshold

# Функция поиска "несчастных" клеток
Ищем "несчастные" клетки и возвращаем массив кортежей их координат
___
**параметры:**
-  `grid` — двумерный массив клеток
- `threshold` — количество необходимых соседей 
___
*возвращает массив кортежей координат "несчастных" клеток*

In [61]:
def get_unhappy_cells(grid, threshold):
    unhappy_cells = []
    for x in range(n):
        for y in range(n):
            if grid[x, y] != 0 and not is_happy(grid, x, y, threshold):
                unhappy_cells.append((x, y))
    return unhappy_cells


# Функция поиска пустых клеток
Ищем пустые клетки и возвращаем массив кортежей их координат
___
**параметры:**
-  `grid` — двумерный массив клеток
___
*возвращает массив кортежей координат пустых клеток*

In [62]:
def get_empty_cells(grid):
    empty_cells = []
    for x in range(n):
        for y in range(n):
            if grid[x, y] == 0:
                empty_cells.append((x, y))
    return empty_cells

# Функция перемещения "несчастных" клеток
Перемещаем "несчастные" клетки в случайные пустые
___
**параметры:**
- `grid` — двумерный массив клеток
- `threshold` — количество необходимых соседей 
___
*возвращает обновленную сетку, где "несчастные" клетки перемещены на новые позиции*

In [63]:
def move_unhappy_cells(grid, threshold):
    unhappy_cells = get_unhappy_cells(grid, threshold)
    empty_cells = get_empty_cells(grid)

    if not unhappy_cells or not empty_cells:
        return grid, True  # нет пустых или несчастных клеток
    
    for (x, y) in unhappy_cells:
        new_x, new_y = random.choice(empty_cells) # случайная пустая клетка
        grid[new_x, new_y] = grid[x, y] # перемещает несчастную клетку
        grid[x, y] = 0  # обнуляем старую клетку
        empty_cells.remove((new_x, new_y))  # удаляем кортеж с координатами пустой клетки
        empty_cells.append((x, y))  # добавляем новый кортеж со старой позиции в качестве пустой

    return grid, False

In [64]:
from PIL import Image
import os
import re

os.makedirs("frames", exist_ok=True)


def plot_grid(grid, step): # визуализация сетки
    plt.figure(figsize=(6, 6))
    plt.imshow(grid, cmap=ListedColormap(['white', 'blue', 'red']), vmin=0, vmax=2)
    plt.title(f"Step {step}")
    plt.colorbar(ticks=[0, 1, 2], label='Cell State')
    filename = f"./frames/plot_step_{step}.png"
    plt.savefig(filename)
    plt.close()

def images_to_gif(directory, gif_name="grid_simulation.gif", duration=100):
    def extract_step(filename):
        match = re.search(r"plot_step_(\d+).png", filename)
        return int(match.group(1)) if match else float('inf')
    
    image_files = sorted(
        [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".png")],
        key=extract_step
    )
    
    frames = [Image.open(image) for image in image_files]
    if frames:
        frames[0].save(
            os.path.join(directory, gif_name),
            save_all=True,
            append_images=frames[1:],
            duration=duration,
            loop=0
        )
    else:
        print("No images found in the directory to create a GIF.")


In [65]:
def simulate_segregation(grid, steps, threshold): # цикл для симуляции
    plot_grid(grid, 0)
    for step in range(1, steps+1):
        grid, stop = move_unhappy_cells(grid, threshold)
        plot_grid(grid, step)
        if stop:
          break

In [66]:
n = 40 # размер сетки (n x n)
empty_ratio = 0.30 # процентное соотношение пустых клеток
blue_ratio = 0.35 # процентное соотношение синих клеток
red_ratio = 0.35 # процентное соотношение красных клеток
threshold = 4 # количество необходимых соседей
steps = 100 # количество шагов моделирования

In [67]:
grid = initialize_grid(n, empty_ratio, blue_ratio, red_ratio)
simulate_segregation(grid, steps, threshold)
images_to_gif("frames", gif_name="grid_simulation.gif", duration=200)

# Итоговый результат

![SegmentLocal](frames/191px-grid_simulation.gif "segment")
